# Time series forecasting with DeepAR
DeepAR is a supervised learning algorithm for forecasting scalar time series. 


Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [1]:
bucket = 'sagemaker-testtimeseries'
prefix = 'sagemaker/test-moredat'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role
import json
import math
from os import path
import pandas as pd
role = get_execution_role()

Now we'll import the Python libraries we'll need.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import time
import json
import glob
import sagemaker.amazon.common as smac
import sagemaker
from sagemaker.predictor import csv_serializer, json_deserializer
import boto3
import  s3fs
import  sagemaker
from sagemaker import get_execution_role

In [3]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()

s3_data_path = "{}/{}/data".format(bucket, prefix)
s3_output_path = "{}/{}/output".format(bucket, prefix)

In [4]:

containers  = {
    'us-west-2': '156387875391.dkr.ecr.us-west-2.amazonaws.com/forecasting-deepar:latest'
}
image_name = containers[boto3.Session().region_name]

In [5]:
role

'arn:aws:iam::945416147148:role/service-role/AmazonSageMaker-ExecutionRole-20180616T095406'

---
## Data

Let's download the data.  More information about this dataset can be found [here](https://rdrr.io/github/robjhyndman/fpp/man/gasoline.html).

In [6]:
#!wget http://gwa.ewi.tudelft.nl/fileadmin/pds/trace-archives/grid-workloads-archive/datasets/gwa-t-12/rnd.zip

In [7]:
#import zipfile
#with zipfile.ZipFile("rnd.zip","r") as zip_ref:
#    zip_ref.extractall("targetdir")

And take a look at it.

# USING FOR MORE DATA - 6/20

In [8]:
files = glob.glob(os.path.join('targetdir/rnd/2013-7', "1*.csv"))
#files_first200 = files[:150]
dfs = [pd.read_csv(fp, sep = ';\t').assign(VM=os.path.basename(fp).split('.')[0]) for fp in files]
df = pd.concat(dfs, ignore_index=True)

files2 = glob.glob(os.path.join('targetdir/rnd/2013-8', "1*.csv"))
#files2_first200 = files2[:150]
dfs2 = [pd.read_csv(fp, sep = ';\t').assign(VM=os.path.basename(fp).split('.')[0]) for fp in files2]
df2 = pd.concat(dfs2, ignore_index=True)

files3 = glob.glob(os.path.join('targetdir/rnd/2013-9', "1*.csv"))
#files3_first200 = files3[:150]
dfs3 = [pd.read_csv(fp, sep = ';\t').assign(VM=os.path.basename(fp).split('.')[0]) for fp in files3]
df3 = pd.concat(dfs3, ignore_index=True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifyin

In [26]:
'''files4 = glob.glob(os.path.join('targetdir/rnd/2013-7', "2*.csv"))
dfs4 = [pd.read_csv(fp, sep = ';\t').assign(VM=os.path.basename(fp).split('.')[0]) for fp in files4]
df4 = pd.concat(dfs4, ignore_index=True)'''

'files4 = glob.glob(os.path.join(\'targetdir/rnd/2013-7\', "2*.csv"))\ndfs4 = [pd.read_csv(fp, sep = \';\t\').assign(VM=os.path.basename(fp).split(\'.\')[0]) for fp in files4]\ndf4 = pd.concat(dfs4, ignore_index=True)'

In [10]:
'''files5 = glob.glob(os.path.join('targetdir/rnd/2013-8', "2*.csv"))
dfs5 = [pd.read_csv(fp, sep = ';\t').assign(VM=os.path.basename(fp).split('.')[0]) for fp in files5]
df5 = pd.concat(dfs5, ignore_index=True)'''

'files5 = glob.glob(os.path.join(\'targetdir/rnd/2013-8\', "2*.csv"))\ndfs5 = [pd.read_csv(fp, sep = \';\t\').assign(VM=os.path.basename(fp).split(\'.\')[0]) for fp in files5]\ndf5 = pd.concat(dfs5, ignore_index=True)'

In [11]:
'''files6 = glob.glob(os.path.join('targetdir/rnd/2013-9', "2*.csv"))
dfs6 = [pd.read_csv(fp, sep = ';\t').assign(VM=os.path.basename(fp).split('.')[0]) for fp in files6]
df6 = pd.concat(dfs6, ignore_index=True)'''

'files6 = glob.glob(os.path.join(\'targetdir/rnd/2013-9\', "2*.csv"))\ndfs6 = [pd.read_csv(fp, sep = \';\t\').assign(VM=os.path.basename(fp).split(\'.\')[0]) for fp in files6]\ndf6 = pd.concat(dfs6, ignore_index=True)'

In [12]:
newdat = df.append(df2)
newerdat = newdat.append(df3)
concatenated_df = newerdat
'''
newdat2 = df.append(df4)
newerdat2 = newdat2.append(df5)
concatenated_df2 = newerdat2

concatenated_df = concatenated_df1.append(concatenated_df2)'''

'\nnewdat2 = df.append(df4)\nnewerdat2 = newdat2.append(df5)\nconcatenated_df2 = newerdat2\n\nconcatenated_df = concatenated_df1.append(concatenated_df2)'

In [13]:
concatenated_df.head()

,Timestamp [ms],CPU cores,CPU capacity provisioned [MHZ],CPU usage [MHZ],CPU usage [%],Memory capacity provisioned [KB],Memory usage [KB],Disk read throughput [KB/s],Disk write throughput [KB/s],Network received throughput [KB/s],Network transmitted throughput [KB/s],VM
0,1372629804,1,2599.999398,41.599990,1.600000,2097152.0,260044.800000,0.400000,7.8,1.666667,7.400000,13
1,1372630104,1,2599.999398,36.399992,1.400000,2097152.0,281016.800000,0.133333,10.0,1.000000,2.000000,13
2,1372630404,1,2599.999398,25.999994,1.000000,2097152.0,209712.000000,0.200000,7.4,1.000000,1.466667,13
3,1372630704,1,2599.999398,31.199993,1.200000,2097152.0,209712.000000,0.066667,6.8,1.000000,1.400000,13
4,1372631004,1,2599.999398,29.466660,1.133333,2097152.0,254453.066667,0.133333,7.0,1.000000,1.466667,13


In [14]:
concatenated_df.tail()

,Timestamp [ms],CPU cores,CPU capacity provisioned [MHZ],CPU usage [MHZ],CPU usage [%],Memory capacity provisioned [KB],Memory usage [KB],Disk read throughput [KB/s],Disk write throughput [KB/s],Network received throughput [KB/s],Network transmitted throughput [KB/s],VM
926201,1380491363,4,11703.997296,150.201299,1.283333,16703488.0,167772.0,0.0,0.066667,0.0,0.0,106
926202,1380491630,4,11703.997296,197.017288,1.683333,16703488.0,100663.2,0.0,1.666667,0.0,0.0,106
926203,1380491663,4,11703.997296,197.017288,1.683333,16703488.0,100663.2,0.0,1.666667,0.0,0.0,106
926204,1380491930,4,11703.997296,146.299966,1.250000,16703488.0,111848.0,0.0,0.000000,0.0,0.0,106
926205,1380491963,4,11703.997296,146.299966,1.250000,16703488.0,111848.0,0.0,0.000000,0.0,0.0,106


In [15]:
concatenated_df['Timestamp'] = pd.to_datetime(concatenated_df['Timestamp [ms]'], unit = 's')
#concatenated_df.apply(pd.to_numeric, errors='ignore')
concatenated_df.describe()
concatenated_df['weekday'] = concatenated_df['Timestamp'].dt.dayofweek


concatenated_df['weekend'] = ((concatenated_df.weekday) // 5 == 1).astype(float)
# Feature engineering with the date
concatenated_df['month']=concatenated_df.Timestamp.dt.month 
concatenated_df['day']=concatenated_df.Timestamp.dt.day
concatenated_df.set_index('Timestamp',inplace=True)
concatenated_df["CPU usage prev"] = concatenated_df['CPU usage [%]'].shift(1)
concatenated_df["CPU_diff"] = concatenated_df['CPU usage [%]'] - concatenated_df["CPU usage prev"]
concatenated_df["received_prev"] = concatenated_df['Network received throughput [KB/s]'].shift(1)
concatenated_df["received_diff"] = concatenated_df['Network received throughput [KB/s]']- concatenated_df["received_prev"]

concatenated_df["transmitted_prev"] = concatenated_df['Network transmitted throughput [KB/s]'].shift(1)
concatenated_df["transmitted_diff"] = concatenated_df['Network transmitted throughput [KB/s]']- concatenated_df["transmitted_prev"]

In [16]:


concatenated_df["start"] = concatenated_df.index
concatenated_df['target'] = concatenated_df['CPU usage [MHZ]']
#jsondat=pd.DataFrame(concatenated_df['start'])
#jsondat['start'] = concatenated_df.index
#result = pd.concat([concatenated_df['start'], concatenated_df['target']], axis=1)
#result = result.reset_index()
#result.drop('Timestamp', axis=1, inplace=True)
#result.start = result.start.astype(str)
#result.set_index('start',inplace=True)
#result

#result.set_index('start',inplace=True)
#result.index = pd.to_datetime(result.index) 


In [17]:

result = pd.concat([concatenated_df['VM'],concatenated_df['start'], concatenated_df['target']], axis=1)
result = result.reset_index()
#result.set_index('start',inplace=True)

result.set_index(['start', 'VM'],inplace=True)
result.drop('Timestamp', axis=1, inplace=True)


In [18]:
df = concatenated_df.groupby('VM').resample('1min', how={'target':np.mean})

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:1502: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  return get_resampler_for_grouping(self, rule, *args, **kwargs)


In [19]:
df.reset_index(level=0, inplace=True)

In [20]:
#df['Timestamp'] = df['Timestamp'].dt.strftime('%m/%d/%Y %H:%M:%S')

In [21]:
df = df.fillna(method='ffill')

In [22]:
freq = "1min"
context_length = 50
prediction_length = 50

In [23]:
dftrain = df[:-(prediction_length)]

In [24]:
dftrain.tail()

,VM,target
Timestamp,,
2013-09-29 21:04:00,199,86.666653
2013-09-29 21:05:00,199,86.666653
2013-09-29 21:06:00,199,86.666653
2013-09-29 21:07:00,199,86.666653
2013-09-29 21:08:00,199,86.666653


In [25]:
df.tail()

,VM,target
Timestamp,,
2013-09-29 21:54:00,199,84.933320
2013-09-29 21:55:00,199,84.933320
2013-09-29 21:56:00,199,84.933320
2013-09-29 21:57:00,199,84.933320
2013-09-29 21:58:00,199,102.266651


In [27]:
def series_to_obj(ts, cat=None):
    obj = {"start": str(ts.index[0]), "target": list(ts)}
    if cat is not None:
        obj["cat"] = cat
    return obj

def series_to_jsonline(ts, cat=None):
    return json.dumps(series_to_obj(ts, cat))

In [28]:
time_series_test=[]
vm_index_range = df['VM'].unique()
for i in vm_index_range:
    newseries = df[df['VM'] == i]['target']
    del newseries.index.name
    newseries.index = pd.to_datetime(newseries.index)
    time_series_test.append(newseries)
    #time_series_test = time_series_test.fillna(0)
    
'''
with open('./test_data.json', 'wb') as fp:
    for ts in time_series_test:
        fp.write(series_to_jsonline(ts).encode(encoding))
        fp.write('\n'.encode(encoding)) '''

"\nwith open('./test_data.json', 'wb') as fp:\n    for ts in time_series_test:\n        fp.write(series_to_jsonline(ts).encode(encoding))\n        fp.write('\n'.encode(encoding)) "

In [29]:
time_series_training=[]
vm_index_range = df['VM'].unique()
for i in vm_index_range:
    newseries = df[df['VM'] == i]['target']
    del newseries.index.name
    newseries.index = pd.to_datetime(newseries.index)
    time_series_training.append(newseries[:-prediction_length])
    #time_series_training = time_series_training.fillna(0)
    


In [30]:
'''with open('./train_data.json', 'wb') as fp:
    for ts in time_series_training:
        fp.write(series_to_jsonline(ts).encode(encoding))
        fp.write('\n'.encode(encoding)) '''

"with open('./train_data.json', 'wb') as fp:\n    for ts in time_series_training:\n        fp.write(series_to_jsonline(ts).encode(encoding))\n        fp.write('\n'.encode(encoding)) "

## can't figure out how to push the json file to S3 bucket... manually moved for now

In [31]:
s3filesystem = s3fs.S3FileSystem()

In [32]:
s3filesystem

In [33]:
encoding = "utf-8"

with s3filesystem.open(s3_data_path + "/test/test_data.json", 'wb') as fp:
    for ts in time_series_test:
        fp.write(series_to_jsonline(ts).encode(encoding))
        fp.write('\n'.encode(encoding)) 

In [34]:
with s3filesystem.open(s3_data_path + "/train/train_data.json", 'wb') as fp:
    for ts in time_series_training:
        fp.write(series_to_jsonline(ts).encode(encoding))
        fp.write('\n'.encode(encoding)) 

In [35]:
s3_data_path

'sagemaker-testtimeseries/sagemaker/test-moredat/data'

In [36]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.xlarge',
    base_job_name='test-demo-deepar',
    output_path="s3://" + s3_output_path
)

In [37]:
# sagemaker automatically adds lags, so context length can be shorter than seasons

In [38]:

hyperparameters  = {
    "time_freq": freq,
    "context_length": context_length,
    "prediction_length": prediction_length,
    "num_cells": "40",
    "num_layers": "3",
    "likelihood": "student-t",
    "epochs": "20",
    "mini_batch_size": "32",
    "learning_rate": "0.001",
    "dropout_rate": "0.05",
    "early_stopping_patience": "10"
}

In [39]:
estimator.set_hyperparameters(**hyperparameters)

In [40]:
s3_data_path

'sagemaker-testtimeseries/sagemaker/test-moredat/data'

In [41]:
data_channels = {
    "train": "s3://{}/train/".format(s3_data_path),
    "test": "s3://{}/test/".format(s3_data_path)
}

estimator.fit(inputs=data_channels)

INFO:sagemaker:Creating training-job with name: test-demo-deepar-2018-06-21-03-55-48-846


......................
Arguments: train
[06/21/2018 03:59:20 INFO 140419976746816] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'dropout_rate': u'0.10', u'cardinality': u'', u'test_quantiles': u'[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'learning_rate': u'0.001', u'num_cells': u'40', u'num_layers': u'2', u'embedding_dimension': u'', u'_kvstore': u'auto', u'_num_kv_servers': u'auto', u'mini_batch_size': u'32', u'likelihood': u'student-t', u'early_stopping_patience': u''}
[06/21/2018 03:59:20 INFO 140419976746816] Reading provided configuration from /opt/ml/input/config/hyperparameters.json: {u'dropout_rate': u'0.05', u'learning_rate': u'0.001', u'num_cells': u'40', u'prediction_length': u'50', u'epochs': u'20', u'time_freq': u'1min', u'context_length': u'50', u'num_layers': u'3', u'mini_batch_size': u'32', u'likelihood': u'student-t', u'early_stopping_patience': 

[06/21/2018 03:59:48 INFO 140419976746816] Epoch[4] Batch[10] avg_epoch_loss=4.349163
[06/21/2018 03:59:48 INFO 140419976746816] Epoch[4] Batch [10]#011Speed: 75.77 samples/sec#011loss=4.407693
[06/21/2018 03:59:48 INFO 140419976746816] processed a total of 368 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 4974.170923233032, "sum": 4974.170923233032, "min": 4974.170923233032}}, "EndTime": 1529553588.822998, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1529553583.835558}

[06/21/2018 03:59:48 INFO 140419976746816] #throughput_metric: host=algo-1, train throughput=73.9807031394 records/second
[06/21/2018 03:59:48 INFO 140419976746816] #progress_metric: host=algo-1, completed 25 % of epochs
[06/21/2018 03:59:48 INFO 140419976746816] best epoch loss so far
[06/21/2018 03:59:48 INFO 140419976746816] Saved checkpoint to "/opt/ml/model/state_de74cd15-c888-4796-98a7-ca4f34bded16-0000.params"
#metrics {"Metrics": {"state.

[06/21/2018 04:00:28 INFO 140419976746816] Epoch[12] Batch[10] avg_epoch_loss=3.990212
[06/21/2018 04:00:28 INFO 140419976746816] Epoch[12] Batch [10]#011Speed: 83.41 samples/sec#011loss=4.127423
[06/21/2018 04:00:28 INFO 140419976746816] processed a total of 338 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 4972.790002822876, "sum": 4972.790002822876, "min": 4972.790002822876}}, "EndTime": 1529553628.531075, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1529553623.544921}

[06/21/2018 04:00:28 INFO 140419976746816] #throughput_metric: host=algo-1, train throughput=67.9685267084 records/second
[06/21/2018 04:00:28 INFO 140419976746816] #progress_metric: host=algo-1, completed 65 % of epochs
[06/21/2018 04:00:28 INFO 140419976746816] best epoch loss so far
[06/21/2018 04:00:28 INFO 140419976746816] Saved checkpoint to "/opt/ml/model/state_65fa3b6a-a1b6-44c8-ad6f-6ee72af60ca8-0000.params"
#metrics {"Metrics": {"stat

[06/21/2018 04:00:59 INFO 140419976746816] Epoch[18] Batch[10] avg_epoch_loss=3.907319
[06/21/2018 04:00:59 INFO 140419976746816] Epoch[18] Batch [10]#011Speed: 78.06 samples/sec#011loss=4.145574
[06/21/2018 04:00:59 INFO 140419976746816] processed a total of 328 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 5136.688947677612, "sum": 5136.688947677612, "min": 5136.688947677612}}, "EndTime": 1529553659.03071, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1529553653.879098}

[06/21/2018 04:00:59 INFO 140419976746816] #throughput_metric: host=algo-1, train throughput=63.8529580615 records/second
[06/21/2018 04:00:59 INFO 140419976746816] #progress_metric: host=algo-1, completed 95 % of epochs
[06/21/2018 04:00:59 INFO 140419976746816] loss did not improve for 1 epochs
[06/21/2018 04:00:59 INFO 140419976746816] Epoch[19] Batch[0] avg_epoch_loss=3.532754
[06/21/2018 04:01:01 INFO 140419976746816] Epoch[19] Batch[5] avg

Create endpoint and predictor
Now that we have trained a model, we can use it to perform predictions by deploying it to an endpoint.

Note: remember to delete the enpoint after running this experiment. A cell at the very bottom of this notebook will do that: make sure you run it at the end.

In [ ]:
job_name = estimator.latest_training_job.name

endpoint_name = sagemaker_session.endpoint_from_job(
    job_name=job_name,
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    deployment_image=image_name,
    role=role
)

INFO:sagemaker:Creating endpoint with name test-demo-deepar-2018-06-21-03-55-48-846


---

In [ ]:
class DeepARPredictor(sagemaker.predictor.RealTimePredictor):

    def set_prediction_parameters(self, freq, prediction_length):
        """Set the time frequency and prediction length parameters. This method **must** be called
        before being able to use `predict`.
        
        Parameters:
        freq -- string indicating the time frequency
        prediction_length -- integer, number of predicted time points
        
        Return value: none.
        """
        self.freq = freq
        self.prediction_length = prediction_length
        
    def predict(self, ts, cat=None, encoding="utf-8", num_samples=100, quantiles=["0.1", "0.5", "0.9"]):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.
        
        Parameters:
        ts -- list of `pandas.Series` objects, the time series to predict
        cat -- list of integers (default: None)
        encoding -- string, encoding to use for the request (default: "utf-8")
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])
        
        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_times = [x.index[-1]+1 for x in ts]
        req = self.__encode_request(ts, cat, encoding, num_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, prediction_times, encoding)
    
    def __encode_request(self, ts, cat, encoding, num_samples, quantiles):
        instances = [series_to_obj(ts[k], cat[k] if cat else None) for k in range(len(ts))]
        configuration = {"num_samples": num_samples, "output_types": ["quantiles"], "quantiles": quantiles}
        http_request_data = {"instances": instances, "configuration": configuration}
        return json.dumps(http_request_data).encode(encoding)
    
    def __decode_response(self, response, prediction_times, encoding):
        response_data = json.loads(response.decode(encoding))
        list_of_df = []
        for k in range(len(prediction_times)):
            prediction_index = pd.DatetimeIndex(start=prediction_times[k], freq=self.freq, periods=self.prediction_length)
            list_of_df.append(pd.DataFrame(data=response_data['predictions'][k]['quantiles'], index=prediction_index))
        return list_of_df

In [ ]:
predictor = DeepARPredictor(
    endpoint=endpoint_name,
    sagemaker_session=sagemaker_session,
    content_type="application/json"
)
predictor.set_prediction_parameters(freq, prediction_length)

In [ ]:
new_time_series_training = []
for ts in time_series_training:
    new_time_series_training.append(ts.asfreq('T'))

In [ ]:
new_time_series_test = []
for ts in time_series_test:
    new_time_series_test.append(ts.asfreq('T'))



In [ ]:
# 2 max for memory (on my desktop)

list_of_df  = predictor.predict(new_time_series_training[:2]) # predicted forecast
actual_data = new_time_series_test # full data set

In [ ]:
for k in range(len(list_of_df)): 
    plt.figure(figsize=(12,6))
    actual_data[k][-prediction_length-context_length:].plot(label='target')
    p10 = list_of_df[k]['0.1'] 
    p90 = list_of_df[k]['0.9'] #set limits predictively
    plt.fill_between(p10.index, p10, p90, color='y', alpha=0.5, label='80% confidence interval')
    list_of_df[k]['0.5'].plot(label='prediction median') # set requests for capacity allocation 
    plt.legend()
    plt.show()

two paths:
** 1. iterating and improving model fit
    a. try to get it to full data set- as big as possible 
    b. get test metric !!! - if large enough use RMSE
    c. use model to make predictions (hold out data or ?)
    
2. look at p90- upper bound in graph = max capacity set in real time, cost calc between this and actual capacity that was provisioned, 

3. extra: try to get in hyperparameter tuning 

4. put final cleaned up notebook

5. accessing data and documentation MAKE CLEAR notes on 

## Created hyperparameter tuning job on the GUI, but attempting to do with code below.

In [ ]:
s3_input_train = 's3://{}/{}/train'.format(bucket, prefix)
s3_input_validation ='s3://{}/{}/validation/'.format(bucket, prefix)

In [706]:
tuning_job_config = {
   "HyperParameterTuningJobConfig": { 
      "HyperParameterTuningJobObjective": { 
         "MetricName": "string",
         "Type": "string"
      },
      "ParameterRanges": { 
         "CategoricalParameterRanges": [ 
            { 
               "Name": "string",
               "Values": [ "string" ]
            }
         ],
         "ContinuousParameterRanges": [ 
            { 
               "MaxValue": "string",
               "MinValue": "string",
               "Name": "string"
            }
         ],
         "IntegerParameterRanges": [ 
            { 
               "MaxValue": "string",
               "MinValue": "string",
               "Name": "string"
            }
         ]
      },
      "ResourceLimits": { 
         "MaxNumberOfTrainingJobs": number,
         "MaxParallelTrainingJobs": number
      },
      "Strategy": "string"
   },
   "HyperParameterTuningJobName": "string",
   "Tags": [ 
      { 
         "Key": "string",
         "Value": "string"
      }
   ],
   "TrainingJobDefinition": { 
      "AlgorithmSpecification": { 
         "MetricDefinitions": [ 
            { 
               "Name": "string",
               "Regex": "string"
            }
         ],
         "TrainingImage": "string",
         "TrainingInputMode": "string"
      },
      "InputDataConfig": [ 
         { 
            "ChannelName": "string",
            "CompressionType": "string",
            "ContentType": "string",
            "DataSource": { 
               "S3DataSource": { 
                  "S3DataDistributionType": "string",
                  "S3DataType": "string",
                  "S3Uri": "string"
               }
            },
            "RecordWrapperType": "string"
         }
      ],
      "OutputDataConfig": { 
         "KmsKeyId": "string",
         "S3OutputPath": "string"
      },
      "ResourceConfig": { 
         "InstanceCount": number,
         "InstanceType": "string",
         "VolumeKmsKeyId": "string",
         "VolumeSizeInGB": number
      },
      "RoleArn": "string",
      "StaticHyperParameters": { 
         "string" : "string" 
      },
      "StoppingCondition": { 
         "MaxRuntimeInSeconds": number
      },
      "VpcConfig": { 
         "SecurityGroupIds": [ "string" ],
         "Subnets": [ "string" ]
      }
   }
}

NameError: name 'number' is not defined

In [704]:
smclient = boto3.Session().client('sagemaker')

In [705]:
smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = tuning_job_name,
                                            HyperParameterTuningJobConfig = tuning_job_config,
                                            TrainingJobDefinition = training_job_definition)


NameError: name 'tuning_job_name' is not defined

### Query a trained model by using the model's endpoint. The endpoint takes the following JSON request format.

In [ ]:

{
 "instances": [
  { "start": "2009-11-01 00:00:00", "target": [4.0, 10.0, 50.0, 100.0, 113.0], "cat": 0},
  { "start": "2012-01-30", "target": [1.0], "cat": 2 },
  { "start": "1999-01-30", "target": [2.0, 1.0], "cat": 1 }
 ],
 "configuration": {
  "num_samples": 50,
  "output_types": ["mean", "quantiles", "samples"],
  "quantiles": ["0.5", "0.9"]
 }
}

In [43]:
sagemaker_session.delete_endpoint(endpoint_name)

NameError: name 'endpoint_name' is not defined